# कार्टपोल सन्तुलन गर्न RL प्रशिक्षण

यो नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) को हिस्सा हो। यसले [PyTorch को आधिकारिक ट्युटोरियल](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) र [यो Cartpole PyTorch कार्यान्वयन](https://github.com/yc930401/Actor-Critic-pytorch) बाट प्रेरणा लिएको छ।

यस उदाहरणमा, हामी RL प्रयोग गरेर एउटा मोडेललाई प्रशिक्षण दिनेछौं जसले एउटा गाडीमा रहेको पोललाई सन्तुलनमा राख्न सक्छ। गाडीले क्षैतिज स्केलमा बायाँ र दायाँ सर्न सक्छ। हामी [OpenAI Gym](https://www.gymlibrary.ml/) वातावरण प्रयोग गरेर पोलको सिमुलेशन गर्नेछौं।

> **Note**: तपाईं यो पाठको कोड स्थानीय रूपमा (जस्तै Visual Studio Code बाट) चलाउन सक्नुहुन्छ, जसमा सिमुलेशन नयाँ विन्डोमा खुल्छ। अनलाइन कोड चलाउँदा, तपाईंले कोडमा केही परिवर्तन गर्नुपर्ने हुन सक्छ, जस्तो [यहाँ](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) वर्णन गरिएको छ।

हामी Gym स्थापना भएको सुनिश्चित गरेर सुरु गर्नेछौं:


In [ ]:
import sys
!{sys.executable} -m pip install gym

अब हामी CartPole वातावरण सिर्जना गरौं र यसमा कसरी काम गर्ने भनेर हेर्नुहोस्। एउटा वातावरणमा निम्न गुणहरू हुन्छन्:

* **Action space** भनेको प्रत्येक चरणको सिमुलेशनमा हामीले गर्न सक्ने सम्भावित क्रियाहरूको सेट हो।  
* **Observation space** भनेको हामीले गर्न सक्ने अवलोकनहरूको क्षेत्र हो।  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

आउनुहोस्, यो सिमुलेशन कसरी काम गर्छ हेर्नुहोस्। तलको लूपले सिमुलेशन चलाउँछ, जबसम्म `env.step` ले समाप्ति संकेत `done` फर्काउँदैन। हामीले `env.action_space.sample()` प्रयोग गरेर क्रियाहरू अनियमित रूपमा चयन गर्नेछौं, जसको अर्थ प्रयोग सम्भवतः धेरै छिटो असफल हुनेछ (CartPole वातावरण समाप्त हुन्छ जब CartPole को गति, यसको स्थिति वा कोण निश्चित सीमाहरू बाहिर हुन्छन्)।

> सिमुलेशन नयाँ झ्यालमा खुल्नेछ। तपाईंले कोडलाई धेरै पटक चलाउन सक्नुहुन्छ र यसको व्यवहार हेर्न सक्नुहुन्छ।


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

तपाईंले देख्न सक्नुहुन्छ कि अवलोकनहरूमा ४ वटा सङ्ख्या छन्। ती हुन्:
- कार्टको स्थिति
- कार्टको वेग
- पोलको कोण
- पोलको घुम्ने दर

`rew` भनेको हरेक चरणमा प्राप्त हुने पुरस्कार हो। CartPole वातावरणमा, तपाईंलाई प्रत्येक सिमुलेशन चरणका लागि १ अंकको पुरस्कार दिइन्छ, र लक्ष्य भनेको कुल पुरस्कारलाई अधिकतम बनाउनु हो, अर्थात् CartPole लाई लड्न नदिई सकेसम्म लामो समयसम्म सन्तुलनमा राख्नु।

रिइन्फोर्समेन्ट लर्निङको क्रममा, हाम्रो लक्ष्य भनेको एउटा **नीति** $\pi$ प्रशिक्षण गर्नु हो, जसले प्रत्येक अवस्था $s$ का लागि कुन क्रिया $a$ लिनुपर्ने हो भन्ने बताउँछ, अर्थात् $a = \pi(s)$।

यदि तपाईंलाई सम्भावनात्मक समाधान चाहिन्छ भने, तपाईं नीतिलाई प्रत्येक क्रियाका लागि सम्भावनाहरूको सेट फिर्ता गर्ने रूपमा सोच्न सक्नुहुन्छ, अर्थात् $\pi(a|s)$ ले $s$ अवस्थामा $a$ क्रिया लिनुपर्ने सम्भावना जनाउँछ।

## नीति ग्रेडियन्ट विधि

सबैभन्दा साधारण RL एल्गोरिदम, जसलाई **नीति ग्रेडियन्ट** भनिन्छ, मा हामी एउटा न्यूरल नेटवर्कलाई अर्को क्रिया भविष्यवाणी गर्न प्रशिक्षण गर्नेछौं।


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

हामी धेरै प्रयोगहरू चलाएर र प्रत्येक रन पछि हाम्रो नेटवर्क अद्यावधिक गरेर नेटवर्कलाई प्रशिक्षण गर्नेछौं। आउनुहोस् एउटा कार्य परिभाषित गरौं जसले प्रयोग चलाउनेछ र परिणामहरू (तथाकथित **ट्रेस**) - सबै अवस्थाहरू, कार्यहरू (र तिनीहरूको सिफारिस गरिएका सम्भावनाहरू), र पुरस्कारहरू फर्काउनेछ:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

तपाईंले अप्रशिक्षित नेटवर्कसँग एक एपिसोड चलाउन सक्नुहुन्छ र कुल पुरस्कार (एपिसोडको लम्बाइ) धेरै कम भएको अवलोकन गर्न सक्नुहुन्छ:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

पोलिसी ग्रेडियन्ट एल्गोरिदमको जटिल पक्षहरू मध्ये एक **छूट गरिएको पुरस्कारहरू** प्रयोग गर्नु हो। विचार यो हो कि हामी खेलको प्रत्येक चरणमा कुल पुरस्कारहरूको भेक्टर गणना गर्छौं, र यस प्रक्रियाको क्रममा हामी केही गुणांक $gamma$ प्रयोग गरेर प्रारम्भिक पुरस्कारहरू छूट गर्छौं। हामी परिणामी भेक्टरलाई पनि सामान्यीकरण गर्छौं, किनकि हामी यसलाई हाम्रो प्रशिक्षणलाई प्रभाव पार्न तौलको रूपमा प्रयोग गर्नेछौं:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

अब हामी वास्तविक प्रशिक्षण सुरु गर्नेछौं! हामी ३०० एपिसोड चलाउनेछौं, र प्रत्येक एपिसोडमा निम्न कार्यहरू गर्नेछौं:

1. प्रयोग चलाउनुहोस् र ट्रेस सङ्कलन गर्नुहोस्।
1. लिएको कार्यहरू र अनुमानित सम्भावनाहरू बीचको फरक (`gradients`) गणना गर्नुहोस्। फरक कम भएमा, हामीले सही कार्य लिएकोमा बढी निश्चित हुन्छौं।
1. छुट्याइएको पुरस्कारहरू गणना गर्नुहोस् र छुट्याइएको पुरस्कारहरूद्वारा gradients लाई गुणा गर्नुहोस् - यसले सुनिश्चित गर्दछ कि उच्च पुरस्कार भएका चरणहरूले अन्तिम नतिजामा कम पुरस्कार भएका चरणहरूको तुलनामा बढी प्रभाव पार्नेछन्।
1. हाम्रो न्यूरल नेटवर्कका लागि अपेक्षित लक्ष्य कार्यहरू आंशिक रूपमा प्रयोगको क्रममा अनुमानित सम्भावनाहरूबाट लिइनेछन्, र आंशिक रूपमा गणना गरिएका gradients बाट। हामी `alpha` प्यारामिटर प्रयोग गरेर gradients र पुरस्कारहरूलाई कति हदसम्म ध्यानमा राख्ने निर्धारण गर्नेछौं - यसलाई reinforcement algorithm को *learning rate* भनिन्छ।
1. अन्ततः, हामी हाम्रो नेटवर्कलाई states र अपेक्षित कार्यहरूमा प्रशिक्षण दिनेछौं, र प्रक्रिया दोहोर्याउनेछौं।


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

अब परिणाम हेर्नको लागि रेन्डरिङसहित एपिसोड चलाउँ।


In [ ]:
_ = run_episode(render=True)

आशा छ, तपाईंले देख्न सक्नुहुन्छ कि पोल अब राम्रोसँग सन्तुलनमा रहन सक्छ!

## अभिनेता-आलोचक मोडेल

अभिनेता-आलोचक मोडेल नीति ग्रेडियन्टको थप विकास हो, जसमा हामी नीतिलाई सिक्न र अनुमानित पुरस्कारहरू सिक्नको लागि न्युरल नेटवर्क निर्माण गर्छौं। यस नेटवर्कमा दुई आउटपुटहरू हुनेछन् (वा तपाईं यसलाई दुई अलग नेटवर्कको रूपमा हेर्न सक्नुहुन्छ):
* **अभिनेता**ले राज्यको सम्भाव्यता वितरण प्रदान गरेर कुन कार्य लिनु पर्ने सिफारिस गर्नेछ, जस्तै नीति ग्रेडियन्ट मोडेलमा।
* **आलोचक**ले ती कार्यहरूबाट के पुरस्कार प्राप्त हुन सक्छ भनेर अनुमान गर्नेछ। यो दिइएको अवस्थामा भविष्यमा कुल अनुमानित पुरस्कारहरू फर्काउँछ।

अब यस्तो मोडेल परिभाषित गरौं:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

हामीले हाम्रो `discounted_rewards` र `run_episode` कार्यहरूलाई अलि परिमार्जन गर्न आवश्यक हुनेछ:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


अब हामी मुख्य प्रशिक्षण लूप चलाउनेछौं। हामी उचित हानिको कार्यहरू गणना गरेर र नेटवर्क प्यारामिटरहरू अद्यावधिक गरेर म्यानुअल नेटवर्क प्रशिक्षण प्रक्रिया प्रयोग गर्नेछौं:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## मुख्य कुरा

यस डेमोमा हामीले दुईवटा RL एल्गोरिदमहरू हेरेका छौं: साधारण पॉलिसी ग्रेडियन्ट र अझ परिष्कृत एक्टर-क्रिटिक। तपाईंले देख्न सक्नुहुन्छ कि ती एल्गोरिदमहरूले अवस्था, क्रिया, र पुरस्कारको अमूर्त अवधारणाहरूसँग काम गर्छन् - त्यसैले तिनीहरू धेरै फरक वातावरणहरूमा लागू गर्न सकिन्छ।

रिइन्फोर्समेन्ट लर्निङले हामीलाई अन्तिम पुरस्कार हेरेर मात्र समस्या समाधान गर्नको लागि सबैभन्दा राम्रो रणनीति सिक्न अनुमति दिन्छ। लेबल गरिएको डेटासेटको आवश्यकता नपर्ने तथ्यले हामीलाई हाम्रो मोडेलहरूलाई अनुकूलन गर्न धेरै पटक सिमुलेसनहरू दोहोर्याउन अनुमति दिन्छ। यद्यपि, RL मा अझै धेरै चुनौतीहरू छन्, जुन तपाईंले यो रोचक AI क्षेत्रमा थप ध्यान केन्द्रित गर्ने निर्णय गर्नुभयो भने सिक्न सक्नुहुन्छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथार्थताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा असमानताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
